In [27]:
# ✅ STEP 1: Install requirements
!pip install llama-cpp-python==0.2.24 PyMuPDF tqdm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 31.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [28]:
!pip install --upgrade llama-cpp-python

  Using cached llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: llama-cpp-python
    Found existing installation: llama_cpp_python 0.2.24
    Uninstalling llama_cpp_python-0.2.24:
      Successfully uninstalled llama_cpp_python-0.2.24


In [29]:
from huggingface_hub import login

# Paste your token when prompted
login()


In [4]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="MaziyarPanahi/Phi-4-mini-instruct-GGUF",
    filename="Phi-4-mini-instruct.Q4_K_S.gguf",
    local_dir="/content/models/phi",
    local_dir_use_symlinks=False
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Phi-4-mini-instruct.Q4_K_S.gguf:   0%|          | 0.00/2.34G [00:00<?, ?B/s]

In [30]:
from llama_cpp import Llama

def load_phi_model(model_path):
    try:
        llm = Llama(
            model_path=model_path,  # Path to your GGUF model
            n_ctx=4096,             # Context window size
            n_threads=8,            # Number of threads
            n_gpu_layers=0,         # Set to 0 if using CPU-only
            use_mmap=False,         # Use memory-mapped file (disabled here)
            verbose=True            # Set to True for verbose output
        )
        return llm
    except Exception as e:
        print("Error loading model:", e)
        return None

model_path = "/content/models/phi/Phi-4-mini-instruct.Q4_K_S.gguf"
model = load_phi_model(model_path)

if model is None:
    print("Model loading failed.")
else:
    print("Model loaded successfully.")


llama_model_loader: loaded meta data with 36 key-value pairs and 196 tensors from /content/models/phi/Phi-4-mini-instruct.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:              phi3.rope.scaling.attn_factor f32              = 1.190238
llama_model_loader: - kv   2:                               general.type str              = model
llama_model_loader: - kv   3:                               general.name str              = Phi 4 Mini Instruct
llama_model_loader: - kv   4:                           general.finetune str              = instruct
llama_model_loader: - kv   5:                           general.basename str              = Phi-4
llama_model_loader: - kv   6:                         general.size_label str              = mini
llama_model_loader: - kv   7:        

Model loaded successfully.


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | BMI2 = 1 | LLAMAFILE = 1 | OPENMP = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'general.quantization_version': '2', 'tokenizer.chat_template': "{% for message in messages %}{% if message['role'] == 'system' and 'tools' in message and message['tools'] is not none %}{{ '<|' + message['role'] + '|>' + message['content'] + '<|tool|>' + message['tools'] + '<|/tool|>' + '<|end|>' }}{% else %}{{ '<|' + message['role'] + '|>' + message['content'] + '<|end|>' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>' }}{% else %}{{ eos_token }}{% endif %}", 'phi3.rope.scaling.original_context_length': '4096', 'phi3.rope.scaling.attn_factor': '1.190238', 'general.architecture': 'phi3', 'general.license': 'mit', 'phi3.context_length': '131072', 'general.type': 'model', 'general.license.link': 'https://huggingface.co/microsoft/Phi-4-mini-instruct/resolve/main/LICENSE', 'tokenizer.ggml.pre': 'gpt-4o', 'general.bas

In [31]:
def load_phi_model(model_path):
    llm = Llama(
    model_path="/content/models/phi/Phi-4-mini-instruct.Q4_K_S.gguf",
    n_ctx=4096,
    n_threads=8,
    n_gpu_layers=0,  # Reduce to 0 if CPU-only
    use_mmap=True,
    verbose=False
)
    return llm


In [11]:
import json
import re
import fitz  # PyMuPDF
from llama_cpp import Llama  # Make sure to install llama-cpp

# Extract clean JSON from output
def extract_json(text):
    text =re.sub(r'^json\s*', '', text.strip())
    try:
        return json.loads(text)
    except Exception as e:
        return {"Error": f"Failed to extract JSON: {str(e)}"}

# Build prompt suited for TinyLLaMA-style chat model
def build_prompt(text):
    instruction = """
You are an information extraction engine. Return ONLY valid JSON, no explanations.

JSON Structure:
{
  "Title": "Paper title",
  "Authors": ["Author 1", "Author 2"],
  "DOI": "DOI if available",
  "Keywords": ["Keyword1", "Keyword2"],
  "Abstract": "Abstract text",
  "Document Type": "Research Paper, Thesis, etc.",
  "Number of References": 10
}

Extract metadata from the following scientific paper:
"""
    return f"<|user|>\n{instruction.strip()}\n{text[:2000]}\n<|assistant|>"


# Call model and extract structured data
def extract_metadata(generator, paper_text):
    prompt = build_prompt(paper_text)
    response = generator(
        prompt,
        max_tokens=1024,             # allow enough room for full JSON
        temperature=0,               # deterministic output
        top_p=1.0,
        stop=["<|end|>", "</s>"],    # prevent model from rambling
    )


    # print(response)
    raw_output = response["choices"][0]["text"]

    print("\n==== RAW MODEL OUTPUT ====")
    print(raw_output)
    print("==== END RAW OUTPUT ====\n")

    return extract_json(raw_output)

# Extract raw text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text if text.strip() else "Error: No extractable text found in PDF."

# Load LLaMA 3 GGUF Model

# Path to your GGUF model
model_path = "/content/models/phi/Phi-4-mini-instruct.Q4_K_S.gguf"
# Load model
model = load_phi_model(model_path)

if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/1.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized



==== RAW MODEL OUTPUT ====
json
{
  "Title": "Extracting Scientific Figures with Distantly Supervised Neural Networks",
  "Authors": ["Noah Siegel", "Nicholas Lourie", "Russell Power", "Waleed Ammar"],
  "DOI": "Not provided",
  "Keywords": ["Figure Extraction", "Distant Supervision", "Deep Learning", "Neural Networks", "Computer Vision"],
  "Abstract": "Non-textual components such as charts, diagrams and tables provide key information in many scientific documents, but the lack of large labeled datasets has impeded the development of data-driven methods for scientific figure extraction. In this paper, we induce high-quality training labels for the task of figure extraction in a large number of scientific documents, with no human intervention. To accomplish this we leverage the auxiliary data provided in two large web collections of scientific documents (arXiv and PubMed) to locate figures and their associated captions in the rasterized PDF. We share the resulting dataset of over 5.5 m

In [12]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/2.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
{
  "Title": "Automatic Recognition of Learning Resource Category in a Digital Library",
  "Authors": ["Soumya Banerjee", "Debarshi Kumar Sanyal", "Samiran Chattopadhyay", "Plaban Kumar Bhowmick", "Partha Pratim Das"],
  "DOI": "Not available",
  "Keywords": ["deep learning", "transfer learning", "digital library"],
  "Abstract": "Digital libraries generally need to process a large volume of diverse document types. The collection and tagging of metadata is a long, error-prone, manpower-consuming task. We are attempting to build an automatic metadata extractor for digital libraries. In this work, we present the Heterogeneous Learning Resources (HLR) dataset for document image classification. The individual learning resource is first decomposed into its constituent document images (sheets) which are then passed through an OCR tool to obtain the textual representation. The document image and its textual content are classified with state-of-the-art classifiers. 

In [13]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/3.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
json
{
  "Title": "Model Selection with Model Zoo via Graph",
  "Authors": ["Ziyu Li", "Hilco van der Wilk", "Danning Zhan", "Megha Khosla", "Alessandro Bozzon", "Rihan Hai"],
  "DOI": "Not provided",
  "Keywords": ["Pre-trained deep learning models", "model zoos", "model selection", "TransferGraph", "graph learning", "model-dataset relationships"],
  "Abstract": "Pre-trained deep learning (DL) models are increasingly accessible in public repositories, i.e., model zoos. Given a new prediction task, finding the best model to fine-tune can be computationally intensive and costly, especially when the number of pre-trained models is large. Selecting the right pre-trained models is crucial, yet complicated by the diversity of models from various model families (like ResNet, Vit, Swin) and the hidden relationships between models and datasets. Existing methods, which utilize basic information from models and datasets to compute scores indicating model performance o

In [14]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/4.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
{
  "Title": "Extracting Decision Model and Notation models from text using deep learning techniques",
  "Authors": ["Alexandre Goossens", "Johannes De Smedt", "Jan Vanthienen"],
  "DOI": "Not available",
  "Keywords": ["Deep learning", "Decision Model and Notation", "DMN", "Decision model extraction"],
  "Abstract": "Companies and organizations often use manuals and guidelines to communicate and execute operational decisions. Decision Model and Notation (DMN) models can be used to model and automate these decisions. Modeling a decision from a textual source, however, is a time intensive and complex activity hence a need for shorter modeling times. This paper studies how NLP deep learning techniques can extract decision models from text faster. In this paper, we study and evaluate an automatic sentence classifier and a decision dependency extractor using NLP deep learning models (BERT and Bi-LSTM-CRF). A large labeled and tagged dataset was collected from re

In [15]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/5.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
{
  "Title": "Automated data extraction is increasingly used to develop databases in materials science and other ﬁelds1. Many databases have been created using automated methods, but there is a growing interest in developing databases for critical cooling rates of metallic glasses and yield strengths of high entropy alloys.",
  "Authors": ["Maciej P. Polak", "Dane Morgan"],
  "DOI": "https://doi.org/10.1038/s41467-024-45914-8",
  "Keywords": ["Automated data extraction", "Materials science", "Critical cooling rates", "Yield strengths", "High entropy alloys"],
  "Abstract": "In this work, we propose the ChatExtract method that can fully automate very accurate data extraction with minimal initial effort and background, using an advanced conversational LLM. ChatExtract consists of a set of engineered prompts applied to a conversational LLM that both identify sentences with data, extract that data, and assure the data’s correctness through a series of follow-up 

In [17]:
import json
import re
import fitz  # PyMuPDF
from llama_cpp import Llama  # Make sure to install llama-cpp

# Extract clean JSON from output
def extract_json(text):
    text =re.sub(r'^json\s*', '', text.strip())
    text = re.sub(r'```json', '', text)
    text = re.sub(r'```', '', text)
    try:
        return json.loads(text)
    except Exception as e:
        return {"Error": f"Failed to extract JSON: {str(e)}"}

# Build prompt suited for TinyLLaMA-style chat model
def build_prompt(text):
    instruction = """
You are an information extraction engine. Return ONLY valid JSON, no explanations.

JSON Structure:
{
  "Title": "Paper title",
  "Authors": ["Author 1", "Author 2"],
  "DOI": "DOI if available",
  "Keywords": ["Keyword1", "Keyword2"],
  "Abstract": "Abstract text",
  "Document Type": "Research Paper, Thesis, etc.",
  "Number of References": 10
}

Extract metadata from the following scientific paper:
"""
    return f"<|user|>\n{instruction.strip()}\n{text[:2000]}\n<|assistant|>"


# Call model and extract structured data
def extract_metadata(generator, paper_text):
    prompt = build_prompt(paper_text)
    response = generator(
        prompt,
        max_tokens=1024,             # allow enough room for full JSON
        temperature=0,               # deterministic output
        top_p=1.0,
        stop=["<|end|>", "</s>"],    # prevent model from rambling
    )


    # print(response)
    raw_output = response["choices"][0]["text"]

    print("\n==== RAW MODEL OUTPUT ====")
    print(raw_output)
    print("==== END RAW OUTPUT ====\n")

    return extract_json(raw_output)

# Extract raw text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text if text.strip() else "Error: No extractable text found in PDF."

# Load LLaMA 3 GGUF Model

# Path to your GGUF model
model_path = "/content/models/phi/Phi-4-mini-instruct.Q4_K_S.gguf"
# Load model
model = load_phi_model(model_path)

if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/6.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))

llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized



==== RAW MODEL OUTPUT ====
```json
{
  "Title": "Smart Learning Environments: Prerequisites-based course recommendation: recommending learning objects using concept prerequisites and metadata matching",
  "Authors": ["Abdessamad Chanaa", "Nour-eddine El Faddouli"],
  "DOI": "https://doi.org/10.1186/s40561-024-00301-0",
  "Keywords": ["Smart Learning Environments", "Prerequisites-based course recommendation", "Learning objects", "Concept prerequisites", "Metadata matching"],
  "Abstract": "The recommendation is an active area of scientific research; it is also a challenging and fundamental problem in online education. However, classical recommender systems usually suffer from item cold-start issues. Besides, unlike other fields like e-commerce or entertainment, e-learning recommendations must ensure that learners have the adequate background knowledge to cognitively receive the recommended learning objects. For that reason, when designing an efficient e-learning recommendation method, 

In [18]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/7.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
```json
{
  "Title": "Deep Transfer Learning for Metadata",
  "Authors": ["Zeyd Boukhers", "Nada Beili", "Timo Hartmann", "Prantik Goswami", "Muhammad Arslan Zafar"],
  "DOI": "Not available",
  "Keywords": ["transfer learning", "metadata extraction", "neural networks"],
  "Abstract": "In contrast to most of the English scientific publications that follow standard and simple layouts, the order, content, position and size of metadata in German publications vary greatly among publications. This variety makes traditional NLP methods fail to accurately extract metadata from these publications. In this paper, we present a method that extracts metadata from PDF documents with different layouts and styles by viewing the document as an image. We used Mask R-CNN which is trained on COCO dataset and ﬁnetuned with PubLayNet dataset that consists of 200K PDF snapshots with ﬁve basic classes (e.g. text, ﬁgure, etc). We reﬁne-tuned the model on our proposed synthetic data

In [19]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/8.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
json
{
  "Title": "ERNIE 3.0: LARGE-SCALE KNOWLEDGE ENHANCED PRE-TRAINING FOR LANGUAGE UNDERSTANDING AND GENERATION",
  "Authors": ["Yu Sun", "Shuohuan Wang", "Shikun Feng", "Siyu Ding", "Chao Pang", "Junyuan Shang", "Jiaxiang Liu", "Xuyi Chen", "Yanbin Zhao", "Yuxiang Lu", "Weixin Liu", "Zhihua Wu", "Weibao Gong", "Jianzhong Liang", "Zhizhou Shang", "Peng Sun", "Wei Liu", "Xuan Ouyang", "Dianhai Yu", "Hao Tian", "Hua Wu", "Haifeng Wang"],
  "DOI": null,
  "Keywords": ["ERNIE 3.0", "Large-scale knowledge enhanced pre-training", "Language understanding", "Language generation", "Zero-shot learning", "Few-shot learning", "Fine-tuning", "Natural Language Processing", "Chinese NLP tasks", "SuperGLUE benchmark"],
  "Abstract": "Pre-trained models have achieved state-of-the-art results in various Natural Language Processing (NLP) tasks. Recent works such as T5 [1] and GPT-3 [2] have shown that scaling up pre-trained language models can improve their generalization 

In [20]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/9.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
```json
{
  "Title": "Few-shot named entity recognition framework for forestry science",
  "Authors": ["Yuquan Fan", "Hong Xiao", "Min Wang", "Junchi Wang", "Wenchao Jiang", "Chang Zhu"],
  "DOI": "https://doi.org/10.1007/s12652-023-04740-4",
  "Keywords": ["metadata extraction", "named entity recognition", "forestry science", "few-shot learning", "data augmentation", "semantic comprehension"],
  "Abstract": "The effective utilization of accumulated forestry science papers is of paramount significance in enhancing our understanding of the current state of forests and the formulation of strategies for forest environmental preservation. However, the present challenge lies in the deficient richness of metadata associated with these pivotal documents, rendering their comprehensive exploitation a formidable endeavor. Metadata from forestry science papers serves as a foundational cornerstone for the efficient management and utilization of these scholarly documents

In [8]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 42.6 MB/s eta 0:00:00


In [11]:
import json
import re
import fitz  # PyMuPDF
from llama_cpp import Llama  # Make sure to install llama-cpp

# Extract clean JSON from output
def extract_json(text):
    text =re.sub(r'^json\s*', '', text.strip())
    text = re.sub(r'```json', '', text)
    text = re.sub(r'```', '', text)
    try:
        return json.loads(text)
    except Exception as e:
        return {"Error": f"Failed to extract JSON: {str(e)}"}

# Build prompt suited for TinyLLaMA-style chat model
def build_prompt(text):
    instruction = """
You are an information extraction engine. Return ONLY valid JSON, no explanations.

JSON Structure:
{
  "Title": "Paper title",
  "Authors": ["Author 1", "Author 2"],
  "DOI": "DOI if available",
  "Keywords": ["Keyword1", "Keyword2"],
  "Abstract": "Abstract text",
  "Document Type": "Research Paper, Thesis, etc.",
  "Number of References": 10
}

Extract metadata from the following scientific paper:
"""
    return f"<|user|>\n{instruction.strip()}\n{text[:2000]}\n<|assistant|>"


# Call model and extract structured data
def extract_metadata(generator, paper_text):
    prompt = build_prompt(paper_text)
    response = generator(
        prompt,
        max_tokens=1024,             # allow enough room for full JSON
        temperature=0,               # deterministic output
        top_p=1.0,
        stop=["<|end|>", "</s>"],    # prevent model from rambling
    )


    # print(response)
    raw_output = response["choices"][0]["text"]

    print("\n==== RAW MODEL OUTPUT ====")
    print(raw_output)
    print("==== END RAW OUTPUT ====\n")

    return extract_json(raw_output)

# Extract raw text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text if text.strip() else "Error: No extractable text found in PDF."

# Load LLaMA 3 GGUF Model

# Path to your GGUF model
model_path = "/content/models/phi/Phi-4-mini-instruct.Q4_K_S.gguf"
# Load model
model = load_phi_model(model_path)
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/10.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))

llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized



==== RAW MODEL OUTPUT ====
```json
{
  "Title": "Extracting structured knowledge from scientific text with large language models",
  "Authors": ["John Dagdelen", "Alexander Dunn", "Sanghoon Lee", "Nicholas Walker", "Andrew S. Rosen", "Gerbrand Ceder", "Kristin A. Persson", "Anubhav Jain"],
  "DOI": "https://doi.org/10.1038/s41467-024-45563-x",
  "Keywords": ["scientific text", "large language models", "joint named entity recognition", "relation extraction", "materials chemistry", "metal-organic frameworks", "composition", "phase", "morphology", "application"],
  "Abstract": "The majority of scientific knowledge about solid-state materials is scattered across the text, tables, and figures of millions of academic research papers. Thus, it is difficult for researchers to properly understand the full body of past work and effectively leverage existing knowledge when designing experiments. Moreover, machine learning models for direct property prediction are being increasingly employed as s

In [12]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/11.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
```json
{
  "Title": "Extracting structured knowledge from scientific text with large language models",
  "Authors": ["John Dagdelen", "Alexander Dunn", "Sanghoon Lee", "Nicholas Walker", "Andrew S. Rosen", "Gerbrand Ceder", "Kristin A. Persson", "Anubhav Jain"],
  "DOI": "https://doi.org/10.1038/s41467-024-45563-x",
  "Keywords": ["scientific text", "large language models", "joint named entity recognition", "relation extraction", "materials chemistry", "metal-organic frameworks", "composition", "phase", "morphology", "application"],
  "Abstract": "Here, we present a simple approach to joint named entity recognition and relation extraction and demonstrate how pretrained large language models (GPT-3, Llama-2) can be fine-tuned to extract useful records of complex scientific knowledge. We test three representative tasks in materials chemistry: linking dopants and host materials, cataloging metal-organic frameworks, and general composition/phase/morphology/appl

In [32]:
import json
import re
import fitz  # PyMuPDF
from llama_cpp import Llama  # Make sure to install llama-cpp

# Extract clean JSON from output
def extract_json(text):
    # Clean up any markdown or prefix like 'json' or '```json'
    text = re.sub(r'^json\s*', '', text.strip(), flags=re.IGNORECASE)
    text = re.sub(r'```json', '', text, flags=re.IGNORECASE)
    text = re.sub(r'```', '', text)
    text = text.encode('utf-8', 'ignore').decode('utf-8', 'ignore')  # remove any weird hidden chars

    try:
        # Parse string into JSON
        metadata = json.loads(text)
        if "Authors" in metadata:
            cleaned_authors = []
            for name in metadata['Authors']:
                # Remove empty parentheses '()', and any other unwanted characters within parentheses
                name = re.sub(r'\(\)', '', name)  # Removes empty parentheses '()'
                name = re.sub(r'\([^)]*\)', '', name)  # Removes content inside parentheses, like (xx)
                cleaned_authors.append(name.strip())  # Append cleaned name

            metadata['Authors'] = cleaned_authors
        return metadata

    except Exception as e:
        return {"Error": f"Failed to extract JSON: {str(e)}"}

# Build prompt suited for TinyLLaMA-style chat model
def build_prompt(text):
    instruction = """
You are an information extraction engine. Return ONLY valid JSON, no explanations.

JSON Structure:
{
  "Title": "Paper title",
  "Authors": ["Author 1", "Author 2"],
  "DOI": "DOI if available",
  "Keywords": ["Keyword1", "Keyword2"],
  "Abstract": "Abstract text",
  "Document Type": "Research Paper, Thesis, etc.",
  "Number of References": 10
}

Extract metadata from the following scientific paper:
"""
    return f"<|user|>\n{instruction.strip()}\n{text[:2000]}\n<|assistant|>"


# Call model and extract structured data
def extract_metadata(generator, paper_text):
    prompt = build_prompt(paper_text)
    response = generator(
        prompt,
        max_tokens=1024,             # allow enough room for full JSON
        temperature=0,               # deterministic output
        top_p=1.0,
        stop=["<|end|>", "</s>"],    # prevent model from rambling
    )


    # print(response)
    raw_output = response["choices"][0]["text"]

    print("\n==== RAW MODEL OUTPUT ====")
    print(raw_output)

    print("==== END RAW OUTPUT ====\n")

    return extract_json(raw_output)

# Extract raw text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text if text.strip() else "Error: No extractable text found in PDF."

# Load LLaMA 3 GGUF Model

# Path to your GGUF model
model_path = "/content/models/phi/Phi-4-mini-instruct.Q4_K_S.gguf"
# Load model
model = load_phi_model(model_path)


if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/12.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))

llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized



==== RAW MODEL OUTPUT ====
{
  "Title": "Information Retrieval Meets Large Language Models: A Strategic Report from Chinese IR Community",
  "Authors": [
    "Qingyao AIa",
    "Ting BAIb",
    "Zhao CAOc",
    "Yi CHANGd",
    "Jiawei CHEN( )e",
    "Zhumin CHENf",
    "Zhiyong CHENGg",
    "Shoubin DONGh",
    "Zhicheng DOUi",
    "Fuli FENG j",
    "Shen GAO f",
    "Jiafeng GUOk",
    "Xiangnan HE( ) j",
    "Yanyan LANa",
    "Chenliang LIl",
    "Yiqun LIUa",
    "Ziyu LYUm",
    "Weizhi MAa",
    "Jun MAf",
    "Zhaochun REN f",
    "Pengjie REN f",
    "Zhiqiang WANGn",
    "Mingwen WANGo",
    "Ji-Rong WENi",
    "Le WUp",
    "Xin XIN f",
    "Jun XUi",
    "Dawei YINq",
    "Peng ZHANG( )r",
    "Fan ZHANGl",
    "Weinan ZHANGs",
    "Min ZHANGa",
    "Xiaofei ZHUt"
  ],
  "DOI": "10.1007/s11740-023-01417-7",
  "Keywords": [
    "Information Retrieval",
    "Large Language Models",
    "Synergistic Relationship",
    "Generative Retrieval",
    "User Understanding",
    "Mo

In [14]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/13.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
{
  "Title": "Building an annotated corpus for automatic metadata extraction from multilingual journal article references",
  "Authors": ["Wonjun Choi", "Hwa-Mook Yoon", "Mi-Hwan Hyun", "Hye-Jin Lee", "Jae-Wook Seol", "Kangsan", "Dajeong Lee", "Young Joon Yoon", "Hyesoo Kong"],
  "DOI": "Not available",
  "Keywords": ["Bibliographic references", "Citation information", "Academic literature", "Metadata extraction", "Multilingual references", "BERT-based transfer-learning model", "GROBID", "CERMINE"],
  "Abstract": "Bibliographic references containing citation information of academic literature play an important role as a medium connecting earlier and recent studies. As references contain machine-readable metadata such as author name, title, or publication year, they have been widely used in the field of citation information services including search services for scholarly information and research trend analysis. Many institutions around the world manually ext

In [16]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/14.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
{
  "Title": "LAME: Layout-Aware Metadata Extraction Approach for Research Articles",
  "Authors": ["JONGYUN CHOI", "HYESOO KONG", "HWAMOOK YOON", "HEUNG-SEON OH", "YUCHUL JUNG"],
  "DOI": "Not provided",
  "Keywords": ["Automatic layout analysis", "Layout-MetaBERT", "Metadata extraction", "Research article"],
  "Abstract": "The volume of academic literature, such as academic conference papers and journals, has increased rapidly worldwide, and research on metadata extraction is ongoing. However, high-performing metadata extraction is still challenging due to diverse layout formats according to journal publishers. To accommodate the diversity of the layouts of academic journals, we propose a novel LAyout-aware Metadata Extraction (LAME) framework equipped with the three characteristics (e.g., design of an automatic layout analysis, construction of a large meta-data training set, and construction of Layout-MetaBERT). We designed an automatic layout analysis us

In [18]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/15.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


==== RAW MODEL OUTPUT ====
json
{
  "Title": "Layout Aware Semantic Element Extraction for Sustainable Science & Technology Decision Support",
  "Authors": ["Hyuntae Kim", "Jongyun Choi", "Soyoung Park", "Yuchul Jung"],
  "DOI": "https://doi.org/10.3390/su14052802",
  "Keywords": ["Layout Aware", "Semantic Element Extraction", "Sustainable Science", "Technology Decision Support", "Knowledge Graph", "Metadata Extraction", "Vision Recognition", "Automated Text Mining"],
  "Abstract": "New scientific and technological (S&T) knowledge is being introduced rapidly, and hence, analysis efforts to understand and analyze new published S&T documents are increasing daily. Automated text mining and vision recognition techniques alleviate the burden somewhat, but the various document layout formats and knowledge content granularities across the S&T ﬁeld make it challenging. Therefore, this paper proposes LA-SEE (LAME and Vi-SEE), a knowledge graph construction framework that simultaneously extract

In [19]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/16.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
```json
{
  "Title": "Convolutional Neural Networks for Sentence Classification",
  "Authors": ["Yoon Kim"],
  "DOI": "Not available",
  "Keywords": ["Convolutional Neural Networks", "Sentence Classification", "Word Vectors", "Deep Learning", "Natural Language Processing"],
  "Abstract": "We report on a series of experiments with convolutional neural networks (CNN) trained on top of pre-trained word vectors for sentence-level classification tasks. We show that a simple CNN with little hyperparameter tuning and static vectors achieves excellent results on multiple benchmarks. Learning task-specific vectors through fine-tuning offers further gains in performance. We additionally propose a simple modification to the architecture to allow for the use of both task-specific and static vectors. The CNN models discussed herein improve upon the state of the art on 4 out of 7 tasks, which include sentiment analysis and question classification.",
  "Document Type": "Re

In [23]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/17.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
{
  "Title": "Information extraction from research papers using conditional random fields",
  "Authors": ["Fuchun Peng", "Andrew McCallum"],
  "DOI": "Not provided",
  "Keywords": ["Information extraction", "Constraint information extraction", "Conditional random fields", "Regularization"],
  "Abstract": "With the increasing use of research paper search engines, such as CiteSeer, for both literature search and hiring decisions, the accuracy of such systems is of paramount importance. This article employs conditional random fields (CRFs) for the task of extracting various common fields from the headers and citation of research papers. CRFs provide a principled way for incorporating various local features, external lexicon features and global layout features. The basic theory of CRFs is becoming well-understood, but best-practices for applying them to real-world data requires additional exploration. We make an empirical exploration of several factors, includin

In [24]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/19.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
```json
{
  "Title": "Predicting movies’ eudaimonic and hedonic scores: A machine learning approach using metadata, audio and visual features",
  "Authors": ["Elham Motamedi", "Danial Khosh Kholgh", "Sorush Saghari", "Mehdi Elahi", "Francesco Barile", "Marko Tkalcic"],
  "DOI": "Unavailable",
  "Keywords": ["Eudaimonia", "Hedonia", "Machine learning approach", "Movie recommender systems"],
  "Abstract": "In the task of modeling user preferences for movie recommender systems, recent research has demonstrated the benefits of describing movies with their eudaimonic and hedonic scores (E and H scores), which reflect the depth of their message and the level of fun experience they provide, respectively. So far, the labeling of movies with their E and H scores has been done manually using a dedicated instrument (a questionnaire), which is time-consuming. To address this issue, we propose an automatic approach for predicting E and H scores. Specifically, we collecte

In [25]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/18.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
json
{
  "Title": "Information extraction from scientific articles: a survey",
  "Authors": ["Zara Nasar", "Syed Waqar Jaffry", "Muhammad Kamran Malik"],
  "DOI": null,
  "Keywords": ["Information extraction", "Scientific articles", "World Wide Web", "Automatic analysis", "Literature review", "Computational approaches", "Rule-based approaches", "Hidden Markov Models", "Conditional Random Fields", "Support Vector Machines", "Naı¨ve-Bayes classification", "Deep Learning"],
  "Abstract": "In last few decades, with the advent of World Wide Web (WWW), world is being overloaded with huge data. This huge data carries potential information that once extracted, can be used for betterment of humanity. Information from this data can be extracted using manual and automatic analysis. Manual analysis is not scalable and efficient, whereas, the automatic analysis involves computing mechanisms that aid in automatic information extraction over huge amount of data. WWW has al

In [26]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/20.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
{
  "Title": "Deep Learning-based Extraction of Algorithmic Metadata in Full-Text Scholarly Documents",
  "Authors": ["Iqra Safdera", "Saeed-Ul Hassana", "Anna Visvizib", "Thanapon Norasetc", "Raheel Nawazd", "Suppawong Tuarobc"],
  "DOI": "Not provided",
  "Keywords": ["Knowledge-based Systems", "Algorithmic Metadata", "Algorithm Search", "Deep Learning", "Bi-Directional LSTM", "Information Retrieval", "Full-text Articles"],
  "Abstract": "The advancements of search engines for traditional text documents have enabled the effective retrieval of massive textual information in a resource-efficient manner. However, such conventional search methodologies often suffer from poor retrieval accuracy especially when documents exhibit unique properties that behoove specialized and deeper semantic extraction. Recently, AlgorithmSeer, a search engine for algorithms has been proposed, that extracts pseudo-codes and shallow textual metadata from scientific publications an

In [8]:
from llama_cpp import Llama

llm = Llama(model_path="/content/models/phi/Phi-4-mini-instruct.Q4_K_S.gguf")

output = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    max_tokens=100,
    temperature=0.7
)

print(output["choices"][0]["message"]["content"])


llama_model_loader: loaded meta data with 36 key-value pairs and 196 tensors from /content/models/phi/Phi-4-mini-instruct.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:              phi3.rope.scaling.attn_factor f32              = 1.190238
llama_model_loader: - kv   2:                               general.type str              = model
llama_model_loader: - kv   3:                               general.name str              = Phi 4 Mini Instruct
llama_model_loader: - kv   4:                           general.finetune str              = instruct
llama_model_loader: - kv   5:                           general.basename str              = Phi-4
llama_model_loader: - kv   6:                         general.size_label str              = mini
llama_model_loader: - kv   7:        

The capital of France is Paris.
